In [1]:
from matminer.featurizers.composition import alloy
from matminer.featurizers.conversions import StrToComposition
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
import sklearn
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
import matplotlib
import joblib
import pandas as pd

In [ ]:
data = pd.read_csv('data.csv')

# Convert formula to composition
data = StrToComposition().featurize_dataframe(data, 'formula')
# 然后基于composition计算特征
data = alloy.WenAlloys().featurize_dataframe(data, 'composition')

In [ ]:
# 去除数据中的'formula', 'C11', 'C12', 'C44', 'a', 'b', 'c', 'G', 'B', 'E', 'v', 'Zener', 'composition', 'Weight Fraction', 'Atomic Fraction'
data.drop(['formula', 'composition', 'Weight Fraction', 'Atomic Fraction'], axis=1, inplace=True)
data.drop(['Interant electrons', 'Interant s electrons', 'Interant p electrons', 'Interant d electrons', 'Interant f electrons'], axis=1, inplace=True)

data.dropna(axis=1, how='any', inplace=True)

# 选择前1500条数据作为训练集和验证集；后500条数据作为验证集。
data_fit = data.iloc[:600]
data_test = data.iloc[600:]

data_fit_X = data_fit[['APE mean', 'Electronegativity local mismatch', 'VEC mean', 'Shear modulus mean', 'Shear modulus delta', 'Shear modulus strength model']]
data_fit_y = data_fit['SFE']
data_test_X = data_test[['APE mean', 'Electronegativity local mismatch', 'VEC mean', 'Shear modulus mean', 'Shear modulus delta', 'Shear modulus strength model']]
data_test_y = data_test['SFE']

In [ ]:
# 参数优化
from multiprocessing import cpu_count, set_start_method

model_xgb = XGBRegressor()
param_grid = {'n_estimators': list(range(100, 1000, 100)), 
              'max_depth': list(range(3, 10, 1)), 
              'learning_rate': ['0.01', '0.1', '0.3', '0.5'], 
              'colsample_bytree': ['0.5', '0.7', '0.9'], 
              'subsample': ['0.5', '0.7', '0.9'], 
              'min_child_weight': [1, 3, 5, 7, 9], 
              'min_split_loss': [0, 1, 3, 5, 7, 9]}
grid_search = GridSearchCV(model_xgb, param_grid, cv=5, n_jobs=cpu_count()-3, verbose=1)

In [ ]:
# 保存模型
import joblib

joblib.dump(grid_search, 'model_GBR.pkl')

# 加载模型
# search_gbr = joblib.load('model_GBR.pkl')